<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

# TP Régression logistique

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from diabeticRetinopathyUtils import load_diabetic_retinopathy
from scipy.optimize import check_grad
from time import time
from sklearn.metrics import classification_report

In [3]:
X, y = load_diabetic_retinopathy("diabeticRetinopathy.csv")
print "Before the insertion:"
print X.shape, y.shape
n, p = X.shape
X = np.c_[np.ones(n), X]
print "After the insertion:"
print X.shape, y.shape

Before the insertion:
(1151, 19) (1151,)
After the insertion:
(1151, 20) (1151,)


In [4]:
def objective(w_, X, y, rho, return_grad=True, return_H=True):
    """
    X: matrix of size n*(p+1)
    y: vector of size n
    w0: real number
    w: vector of size p
    """
    # Initialize elementary intermediate variables;
    n, p = X.shape
    w = w_[1:]
    y_x = np.array([y[i] * X[i, :] for i in range(n)])
    yx_w = np.array([np.sum(y_x[i, :] * w_) for i in range(n)])
    exp_yxw_1 = np.array([np.exp(yx_w[i]) for i in range(n)]) + 1
    exp_neg_yxw_1 = np.array([np.exp(-yx_w[i]) for i in range(n)]) + 1

    # Compute function value
    val = np.mean(np.log(exp_neg_yxw_1)) + np.sum(w**2) * rho / 2.

    if return_grad == False:
        return val
    else:
        # Compute gradient
        grad = np.mean(-np.array([y_x[i] / exp_yxw_1[i]
                                  for i in range(n)]), axis=0) + rho * np.r_[0, w]

        if return_H == False:
            return val, grad
        else:
            # Compute the Hessian matrix
            H = np.mean(np.array([y_x[i].reshape(-1, 1).dot(y_x[i].reshape(1, -1) / (exp_yxw_1[i] * exp_neg_yxw_1[i]))
                                  for i in range(n)]), axis=0) + rho * np.diag(np.r_[0, np.ones(p - 1)])
            return val, grad, H

In [5]:
def funcMask(w_, X, y, rho):
    val, grad = objective(w_, X, y, rho, return_H=False)
    return val


def gradMask(w_, X, y, rho):
    val, grad = objective(w_, X, y, rho, return_H=False)
    return grad


rho = 1. / n
t0 = time()
print "The difference of gradient is: %0.12f" % check_grad(funcMask, gradMask, np.zeros(p + 1), X, y, rho)
print "Done in %0.3fs." % (time() - t0)

The difference of gradient is: 0.000000036693
Done in 0.341s.


In [6]:
def gradMask(w_, X, y, rho):
    val, grad = objective(w_, X, y, rho, return_H=False)
    return grad.sum()


def hessianMask(w_, X, y, rho):
    val, grad, H = objective(w_, X, y, rho)
    return np.sum(H, axis=1)


t0 = time()
rho = 1. / n
print "The difference of Hessian matrix is: %0.12f" % check_grad(gradMask, hessianMask, np.zeros(p + 1), X, y, rho)
print "Done in %0.3fs." % (time() - t0)

The difference of Hessian matrix is: 0.000000144496
Done in 0.448s.


In [10]:
def val_proximal(w_, X, y, rho):
    """
    X: matrix of size n*(p+1)
    y: vector of size n
    w: vector of size p
    """
    # Initialize elementary intermediate variables;
    n, p = X.shape
    w = w_[1:]
    y_x = np.array([y[i] * X[i, :] for i in range(n)])
    yx_w = np.array([np.sum(y_x[i, :] * w_) for i in range(n)])
    exp_neg_yx_w = np.array([np.exp(-yx_w[i]) for i in range(n)]) + 1

    # Compute function value
    val = np.mean(np.log(exp_neg_yx_w)) + rho * np.sum(np.fabs(w))
    return val


def func(w_, X, y, return_grad=True):
    """
    X: matrix of size n*(p+1)
    y: vector of size n
    w: vector of size p
    """
    # Initialize elementary intermediate variables;
    n, p = X.shape
    w = w_[1:]
    y_x = np.array([y[i] * X[i, :] for i in range(n)])
    yx_w = np.array([np.sum(y_x[i, :] * w_) for i in range(n)])
    exp_yx_w = np.array([np.exp(yx_w[i]) for i in range(n)]) + 1
    exp_neg_yx_w = np.array([np.exp(-yx_w[i]) for i in range(n)]) + 1

    # Compute function value
    val = np.mean(np.log(exp_neg_yx_w))

    if return_grad == False:
        return val
    else:
        # Compute gradient
        grad = np.mean(-np.array([y_x[i] / exp_yx_w[i]
                                  for i in range(n)]), axis=0)
        return val, grad


def soft_Threshold(w, rho):
    w_ = np.zeros_like(w)
    w_[w > rho] = w[w > rho] - rho
    w_[w < -rho] = w[w < -rho] + rho
    w_[0] = w[0]
    return w_


def minimize_prox_grad_Taylor(func,
                              f,
                              w_,
                              X,
                              y,
                              rho,
                              a,
                              b,
                              tol=1e-10,
                              max_iter=500):
    n, p = X.shape
    val = func(w_, X, y, rho)
    val_f, grad_f = f(w_, X, y)
    gamma = b / 2.
    delta_val = tol * 2
    cnt = 0
    while (delta_val > tol and cnt < max_iter):
        gamma = 2 * gamma
        w_new = Soft_Threshold(w_ - gamma * grad_f, gamma * rho)
        val_f_ = f(w_new, X, y, return_grad=False)
        # while (val_f_ > val_f + beta*np.sum(grad_f*(w_new - w_))):
        while (val_f_ > val_f + np.sum(grad_f * (w_new - w_)) + np.sum(
                (w_new - w_)**2) / gamma):
            # print val_
            gamma = gamma * a
            w_new = soft_Threshold(w_ - gamma * grad_f, gamma * rho)
            val_f_ = f(w_new, X, y, return_grad=False)
        w_ = w_new
        val_f, grad_f = f(w_, X, y)
        val_ = func(w_, X, y, rho)
        delta_val = val - val_
        val = val_
        cnt = cnt + 1
    return func(w_, X, y, rho), w_, cnt


t0 = time()
rho = 0.1
a = 0.5
b = 1
val_pgls, w_pgls, cnt_pgls = minimize_prox_grad_Taylor(
    objective_proximal,
    f,
    0.3 * np.ones(p + 1),
    X,
    y,
    rho,
    a,
    b,
    tol=1e-8,
    max_iter=500)
print "The value minimal of the objective function is: %0.12f" % val_pgls
t_pgls = time() - t0
print "Done in %0.3fs, number of iterations: %d" % (t_pgls, cnt_pgls)
print w_pgls

The value minimal of the objective function is: 0.686959525572
Done in 6.463s, number of iterations: 126
[ 0.12293998  0.          0.          0.18833666  0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.        ]
